In [3]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [6]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df_home_sales = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

df_home_sales.createOrReplaceTempView('home_sales')

In [11]:
df_home_sales.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [12]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
result = spark.sql("""
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY YEAR(date)
""")

# Displaying the result
result.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [16]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

result = spark.sql("""
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY YEAR(date)
    ORDER BY YEAR(date)
""")

# Displaying the result
result.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|    287287.82|
|2020|    294204.16|
|2021|    294211.46|
|2022|    292725.69|
+----+-------------+



In [18]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?


result = spark.sql("""
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_lot >= 2000
    GROUP BY YEAR(date)
    ORDER BY YEAR(date)
""")

# Displaying the result
result.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|    282752.72|
|2020|    293176.41|
|2021|    290319.06|
|2022|    287162.18|
+----+-------------+



In [19]:
df_home_sales.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [20]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

result = spark.sql("""
    SELECT view AS view, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
""")

# Displaying the result
result.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|   7|    403005.77|
|  51|    788128.21|
|  15|     404673.3|
|  54|    798684.82|
|  11|    399548.12|
|  29|    397771.65|
|  69|    750537.94|
|  42|     396964.5|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|   3|     398867.6|
|  30|     397862.0|
|  34|    401419.75|
|  59|     791453.0|
|   8|    398592.71|
|  28|    402124.62|
|  22|    402022.68|
|  85|   1056336.74|
|  16|    399586.53|
+----+-------------+
only showing top 20 rows

--- 0.8671326637268066 seconds ---


In [21]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [22]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [23]:
df_home_sales.columns

['id',
 'date',
 'date_built',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view']

In [32]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

result = spark.sql("""
    SELECT view AS view, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
""")

# Displaying the result
result.show()




print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|   7|    403005.77|
|  51|    788128.21|
|  15|     404673.3|
|  54|    798684.82|
|  11|    399548.12|
|  29|    397771.65|
|  69|    750537.94|
|  42|     396964.5|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|   3|     398867.6|
|  30|     397862.0|
|  34|    401419.75|
|  59|     791453.0|
|   8|    398592.71|
|  28|    402124.62|
|  22|    402022.68|
|  85|   1056336.74|
|  16|    399586.53|
+----+-------------+
only showing top 20 rows

--- 0.4444618225097656 seconds ---


In [33]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df_home_sales.write.parquet('parquet_home_sales', mode='overwrite')

In [34]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('parquet_home_sales')

In [35]:
# 12. Create a temporary table for the parquet data.
df_home_sales.createOrReplaceTempView('parquet_home_sales')

In [37]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""select date_built,
round(AVG(price),2) as average_price
from parquet_home_sales 
where bedrooms=3 and bathrooms=3 and floors=2 and sqft_living>=2000
group by date_built
order by date_built asc
"""
).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+

--- 0.5515890121459961 seconds ---


In [38]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [39]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')

False